In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.core.prompts.prompts import SimpleInputPrompt

In [2]:
documents= SimpleDirectoryReader("data").load_data()
print(len(documents))
print(documents[0])

64
Doc ID: 61814818-0e78-475c-b5d1-dbe126400646
Text: Published as a conference paper at ICLR 2023 REAC T: S
YNERGIZING REASONING AND ACTING IN LANGUAGE MODELS Shunyu Yao∗*,1,
Jeffrey Zhao2, Dian Yu2, Nan Du2, Izhak Shafran2, Karthik Narasimhan1,
Yuan Cao2 1Department of Computer Science, Princeton University
2Google Research, Brain team 1{shunyuy,karthikn}@princeton.edu
2{jeffreyzhao,dianyu,dunan,...


In [3]:
system_prompt=""" 
You are a knowledgeable and smart Q&A assistant.

Your goals:
- Use the provided context or retrieved documents to give accurate and relevant answers.
- If the context doesn’t contain the exact answer, say you don’t have enough information instead of guessing.
- Always summarize clearly and concisely.
- When possible, provide structured and easy-to-read responses (use bullet points or short paragraphs).
- Do not include unrelated or speculative content.
- If the user’s question is ambiguous, politely ask for clarification.
- Maintain a professional yet approachable tone.

Example style:
User: "What is LlamaIndex?"
Assistant: "LlamaIndex is a data framework that helps you connect external data (like files, databases, or APIs) to large language models. It simplifies loading, indexing, and querying data."

You always follow these principles for every response.
"""

# default format supportable by llama2
query_wrapper_prompt = SimpleInputPrompt(f"<|USER|>{system_prompt}<|ASSISTANT|>")

In [4]:
from dotenv import load_dotenv
load_dotenv()

import os
from typing import List, Optional
HF_TOKEN: Optional[str] = os.getenv("HUGGINGFACE_API_KEY")

### **Now we set the settings for entire system**

In [9]:
from llama_index.llms.huggingface_api import HuggingFaceInferenceAPI
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import Settings

Settings.llm = HuggingFaceInferenceAPI(
    model_name="deepseek-ai/DeepSeek-R1-0528",
    token=HF_TOKEN,
    provider="together",  # this will use the best provider available
)
Settings.embed_model = HuggingFaceEmbedding(model_name="sentence-transformers/all-MiniLM-L6-v2")
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=20)
Settings.context_window = 4096

2025-11-04 14:22:58,657 - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


In [10]:
# a vector store index only needs an embed model
index = VectorStoreIndex.from_documents(
    documents, embed_model=Settings.embed_model, show_progress=True
)

Generating embeddings: 100%|██████████| 163/163 [00:00<00:00, 193.73it/s]


In [11]:
from prompt_toolkit import prompt


query_engine=index.as_query_engine(
    prompt=query_wrapper_prompt
)

In [12]:
result = query_engine.query("What is YOLO?")
print(result.response)

<think>
Hmm, the user is asking about YOLO based on a research paper context. Let me analyze the provided text carefully.

The context describes YOLO as a real-time object detection system that processes streaming video with under 25ms latency. It mentions three key advantages: superior speed/accuracy ratio compared to other real-time systems, global image reasoning that reduces background errors, and strong generalization to new domains like artwork.

The technical section explains YOLO's unified approach - using a single neural network to predict all bounding boxes simultaneously across an S×S grid. Each grid cell predicts bounding boxes with confidence scores based on object presence and localization accuracy.

I should highlight these core aspects: real-time capability, unified detection framework, grid-based prediction system, and its advantages over methods like R-CNN. The answer should synthesize these points without referencing the context directly.

The response needs to be co